# Importing packages and data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import string
import seaborn as sns
import re

from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download("all")

In [ ]:
# Get the review data
healthygym_trustpilot_df = pd.read_excel('/content/sample_data/Trustpilot_12_months.xlsx')
healthygym_google_df = pd.read_excel('/content/sample_data/Google_12_months.xlsx')

In [ ]:
# Trustpilot dataset info
healthygym_trustpilot_df.info()

No null value found on 'Review Content' column

In [ ]:
# Trustpilot dataset sample
healthygym_trustpilot_df[["Location Name", "Review Created (UTC)", "Review Content", "Review Stars"]].head()

In [ ]:
# Google dataset info
healthygym_google_df.info()

In [ ]:
# Only Comment column has some nulls. Can safely run dropna on the dataframe to drop rows with no comments
healthygym_google_df.dropna(inplace=True)

healthygym_google_df.info()

In [ ]:
# Google dataset sample data
healthygym_google_df[["Club's Name", "Creation Date", "Comment", "Overall Score"]].head()

# Conducting initial data investigation

In [ ]:
#get unique locations from Trustpilot reviews
trustpilot_locations = healthygym_trustpilot_df['Location Name'].unique()
trustpilot_locations = trustpilot_locations.astype(str)
len(trustpilot_locations)

In [ ]:
#get unique locations from Google reviews
google_locations = healthygym_google_df["Club's Name"].unique()
google_locations = google_locations.astype(str)
len(google_locations)

In [ ]:
#Get locations common in both datasets
intersection_of_locations = [ tp_location for tp_location in trustpilot_locations if tp_location in google_locations ]
len(intersection_of_locations)

310 locations overlap between google and trust pilot

In [ ]:
# Define stop words and punctuation.
stop_words = set(stopwords.words('english'))

excludes = ["healthygym", "pure", "gym", "one"]

lemma = WordNetLemmatizer()

# Function to clean the document.
def word_cloud_display(df, key):
  top_number_of_results = 100
  all_reviews = df[key].str.lower().str.cat(sep=' ')
  all_text = re.sub('[^A-Za-z]+', ' ', all_reviews)

  word_tokens = word_tokenize(all_text)

  filtered_all_reviews = [lemma.lemmatize(word) for word in word_tokens if (word not in stop_words) and not(word.isnumeric()) and (word not in excludes)]

  word_distribution = nltk.FreqDist(filtered_all_reviews)
  reviews_word_frequency_distribution_df = pd.DataFrame(word_distribution.most_common(top_number_of_results), columns=['Word', 'Frequency'])

  plt.figure(figsize=(8,8))
  sns.set_style("whitegrid")
  ax = sns.barplot(x="Word",y="Frequency", data=reviews_word_frequency_distribution_df.head(10))

  plt.figure(figsize = (60,60))
  wc = WordCloud(background_color = 'black', max_words = 1000,  max_font_size = 50)
  wc.generate(' '.join(filtered_all_reviews))
  plt.imshow(wc)
  plt.axis('off')

In [ ]:
# Clean and display histogram and word cloud - Trustpilot reviews
word_cloud_display(healthygym_trustpilot_df, "Review Content")

In [ ]:
# Clean and display histogram and word cloud - Google reviews
word_cloud_display(healthygym_google_df, "Comment")

The top 10 most frequently used words seem to be very similar from both Google review and Trustpilot.

In [ ]:
# Clean and display histogram and word cloud - Trustpilot negative reviews
bad_healthygym_trustpilot_df = healthygym_trustpilot_df[healthygym_trustpilot_df["Review Stars"] < 3]
word_cloud_display(bad_healthygym_trustpilot_df, "Review Content")

In [ ]:
# Clean and display histogram and word cloud - Google negative reviews

bad_healthygym_google_df = healthygym_google_df[healthygym_google_df["Overall Score"] < 3]
word_cloud_display(bad_healthygym_google_df, "Comment")

The top 10 most frequently used words from negative reviews also seem to be very similar from both Google review and Trustpilot.

# Conducting initial topic modelling

In [ ]:
# Filter out the reviews that are from the locations common to both data sets.
both_loc_healthygym_trustpilot_df = bad_healthygym_trustpilot_df[bad_healthygym_trustpilot_df['Location Name'].isin(intersection_of_locations)]
both_loc_healthygym_google_df = bad_healthygym_google_df[bad_healthygym_google_df["Club's Name"].isin(intersection_of_locations)]

bad_healthygym_trustpilot_list = both_loc_healthygym_trustpilot_df["Review Content"].to_list()
bad_healthygym_google_list = both_loc_healthygym_google_df["Comment"].to_list()

#Merge the reviews to form a new list.
all_bad_reviews_list = bad_healthygym_trustpilot_list + bad_healthygym_google_list

len(all_bad_reviews_list)

In [ ]:
all_bad_reviews_list[0]

In [ ]:
# install BERTopic
!pip install bertopic

In [ ]:
# import pipeline and BERTopic
from transformers import pipeline
from bertopic import BERTopic

In [ ]:
#define function to clean review text
def clean_review_text(text):
    text = text.lower()
    text = re.sub('[^A-Za-z]+', ' ', text)

    word_tokens = word_tokenize(text)

    for word in word_tokens:
      if ((word in stop_words) or word.isnumeric() or (word in excludes)):
        text = text.replace(" " + word + " ", " ")
      else:
        word_lemma = lemma.lemmatize(word)
        text = text.replace(word, word_lemma)
    return text

#define function to clean review text
def clean_reviews(reviews_list):
  reviews_list_clean = []
  for review in reviews_list:
    #print(review)
    if review is not None and review != "nan":
      review =  clean_review_text(review)
      reviews_list_clean.append(review)

  return reviews_list_clean

all_bad_reviews_list_clean = clean_reviews(all_bad_reviews_list)

In [ ]:
all_bad_reviews_list_clean[0]

In [ ]:
# Analyse negative reviews with BERTopic
model = BERTopic(verbose=True)
model.fit(all_bad_reviews_list_clean)
topic, probabilities = model.transform(all_bad_reviews_list_clean)

In [ ]:
# Diplay top frequency topics
model.get_topic_freq().head(10)

In [ ]:
model.get_topic(0)

In [ ]:
model.get_topic(1)

In [ ]:
model.visualize_topics()

In [ ]:
model.visualize_barchart(topics=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
model.visualize_heatmap()

The top 10 clusters are as follows:

1. Shower facility: Shower facility needs to be maintained. Words such as shower, cold, mould, dirty indicates they may need some maintenance.
2. Air quality and freshness: Words such as aircon, air conditioning, hot indicates air quality in the gym needs to be looked at and ensure appropriate ventilation and air conditioners are in correct working order.
3. Classes and Instructors: There seems to be low availability of classes and instructors. class, instructor, booked, cancelled, spin - indicates class cancellations.
4. Access issue: Pass, pin, code, and access indicate there is issue with the access.
5. Parking. There seems to be demand for free parking and instances of customers being fined for parking. Parking, car, park, fine, free are words indicating parking concerns.
6. Equipment and weights: Weights, bench, dumbbells and plates indicates appropriate and sufficient equipment need to be provided.
7. Toilets and changing rooms: toilets, dirty, changing, disgusting indicates toilets and changing room cleanliness is a big concern
8. Music and ambience: There are reviews about loud music and other noise.
9. Membership and cancellation: Membership, account, suspended, month etc indicates issue with user account maintenance.
10. Locker room issue. There seems to be safety issues in the locker room. Words such as Locker, bag, stolen, recording etc indicates there may be some serious issues that need to be rectified with urgency.

  

# Performing further data investigation

In [ ]:
#List out the top 20 locations with the highest number of negative reviews - Trustpilot
bad_healthygym_trustpilot_grouped_df = bad_healthygym_trustpilot_df.groupby('Location Name')['Review Content'].count() #
bad_healthygym_trustpilot_grouped_df = bad_healthygym_trustpilot_grouped_df.sort_values(ascending=False)
bad_healthygym_trustpilot_grouped_df = bad_healthygym_trustpilot_grouped_df.reset_index(name="total_count_tp")

#Get top 20 locations
bad_healthygym_trustpilot_grouped_df.head(20)

In [ ]:
#List out the top 20 locations with the highest number of negative reviews - Google
bad_healthygym_google_grouped_df = bad_healthygym_google_df.groupby("Club's Name")['Comment'].count()
bad_healthygym_google_grouped_df = bad_healthygym_google_grouped_df.sort_values(ascending=False)
bad_healthygym_google_grouped_df = bad_healthygym_google_grouped_df.reset_index(name='total_count_g')

#Get top 20 locations
bad_healthygym_google_grouped_df.head(20)

In [ ]:
# top 20 location with highest negative review - Google
bad_healthygym_google_grouped_df.head(20)["Club's Name"]

In [ ]:
intersection_of_bad_locations = [ tp_bad_location
                                 for tp_bad_location in bad_healthygym_trustpilot_grouped_df.head(20)['Location Name'].to_list()
                                if tp_bad_location in bad_healthygym_google_grouped_df.head(20)["Club's Name"].to_list() ]
len(intersection_of_bad_locations)

There are some common locations (7 locations) but majority are different.

In [ ]:
# Top 20 locations from combined bad reviews
merged_location_bad_reviews_count_df = pd.merge(bad_healthygym_trustpilot_grouped_df, bad_healthygym_google_grouped_df, how='left', left_on='Location Name', right_on="Club's Name")
merged_location_bad_reviews_count_df = merged_location_bad_reviews_count_df.dropna()
merged_location_bad_reviews_count_df.drop("Club's Name", axis=1, inplace=True)
merged_location_bad_reviews_count_df['total_count'] = merged_location_bad_reviews_count_df['total_count_tp'] + merged_location_bad_reviews_count_df['total_count_g']
merged_location_bad_reviews_count_df_sorted = merged_location_bad_reviews_count_df.sort_values(by=['total_count'], ascending=False)
print(len(merged_location_bad_reviews_count_df_sorted))
merged_location_bad_reviews_count_df_sorted.head(20)

In [ ]:
# Get negative reviews from top 30 locations
top30_bad_review_locations = merged_location_bad_reviews_count_df_sorted.head(30)['Location Name']

top30_bad_review_trustpilot_df = bad_healthygym_trustpilot_df[bad_healthygym_trustpilot_df['Location Name'].isin(top30_bad_review_locations)]
top30_bad_review_google_df = bad_healthygym_google_df[bad_healthygym_google_df["Club's Name"].isin(top30_bad_review_locations)]

In [ ]:
# display histogram and word cloud
word_cloud_display(top30_bad_review_trustpilot_df, "Review Content")

In [ ]:
# display histogram and word cloud
word_cloud_display(top30_bad_review_google_df, "Comment")

**Overall bad review analysis**

Trustpilot

*   time
*   equipment
*   membership
*   get
*   machine
*   member
*   staff
*   class
*   day
*   use

Google

*   equipment
*   machine
*   time
*   staff
*   people
*   get
*   like
*   member
*   weight
*   even

**Top 30 bad review analysis**

Trustpilot

*   member
*   machine
*   shower
*   time
*   equipment
*   people
*   also
*   staff
*   toilet
*   month

Google

*   machine
*   equipment
*   time
*   staff
*   people
*   always
*   get
*   even
*   member
*   place

Comparing the word frequency and word cloud between overall negative reviews and top 30 location negative reviews, there are higher number of common words than distinct words. Comparison between Trustpilot and Google reviews also showed similar conclusion indicating that the negative reviews are consistent across both platforms.

In [ ]:
# Filter out the reviews that are from the top 30 bad review locations.
top30_bad_review_trustpilot_list = top30_bad_review_trustpilot_df["Review Content"].to_list()
top30_bad_review_google_list = top30_bad_review_google_df["Comment"].to_list()

#Merge the reviews to form a new list.
top30_bad_reviews_list = top30_bad_review_trustpilot_list + top30_bad_review_google_list

len(top30_bad_reviews_list)

In [ ]:
top30_bad_reviews_list[0]

In [ ]:
top30_bad_reviews_list_clean = clean_reviews(top30_bad_reviews_list)

In [ ]:
top30_bad_reviews_list_clean[0]

In [ ]:
# analyse negative reviews from top 30 locations
model_top30 = BERTopic(verbose=True)
model_top30.fit(top30_bad_reviews_list_clean)
topic_top30, probabilities_top30 = model_top30.transform(top30_bad_reviews_list_clean)

In [ ]:
model_top30.get_topic_freq().head(10)

In [ ]:
model_top30.get_topic(0)

In [ ]:
model_top30.get_topic(1)

In [ ]:
model_top30.visualize_topics()

In [ ]:
model_top30.visualize_barchart(topics=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
model_top30.visualize_heatmap()

The results are almost exactly the same as the first run of BERTopic. Therefore it deduces that if the top 10 to 15 issues are resolved, that would address most of the issues the gym is facing.

Conversely, as these negative reviews are from top 30 locations with most negative reviews, it could also be overlooking negative reviews from other locations.

# Conducting emotion analysis

In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer

# Load the tokenizer associated with the BERT model
tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")

# Load the text classification pipeline with bhadresh-savani/bert-base-uncased-emotion from Hugging Face
classifier = pipeline(
    "text-classification",
    model="bhadresh-savani/bert-base-uncased-emotion",
    tokenizer=tokenizer,
    truncation=True,
    return_all_scores=False)

In [ ]:
# define method to classify reviews
def classify_review(review, max_length=512):
    if isinstance(review, str):

        result = classifier(review)[0]

        # Return the label with the highest score
        return result['label'], result['score']
    else:
        raise ValueError("Input must be a string.")


print(classify_review( top30_bad_reviews_list_clean[1]))

In [ ]:
# get negative reviews in dataframe
healthygym_trustpilot_reviews = bad_healthygym_trustpilot_df["Review Content"].to_frame()
healthygym_google_reviews = bad_healthygym_google_df["Comment"].to_frame()

In [ ]:
healthygym_trustpilot_reviews['emotion_label'], healthygym_trustpilot_reviews['emotion_score'] = zip(*healthygym_trustpilot_reviews['Review Content'].apply(classify_review))

# Display the dataset with the new columns
print(healthygym_trustpilot_reviews.head())

In [ ]:
#Group reviews by emotions - Trustpilot
healthygym_trustpilot_reviews_groupby_emotion_df = healthygym_trustpilot_reviews.groupby('emotion_label')['emotion_label'].count() #
healthygym_trustpilot_reviews_groupby_emotion_df = healthygym_trustpilot_reviews_groupby_emotion_df.sort_values(ascending=False)
healthygym_trustpilot_reviews_groupby_emotion_df = healthygym_trustpilot_reviews_groupby_emotion_df.reset_index(name="total_count")

# Get top 20 emotions
healthygym_trustpilot_reviews_groupby_emotion_df.head(20)

In [ ]:
# display the emotion histogram
ax = healthygym_trustpilot_reviews_groupby_emotion_df.plot.bar(x='emotion_label', y='total_count', rot=0)
ax.set_xlabel("emotion")
ax.set_ylabel("count")
ax.set_title("Predicted Emotions in Negative Reviews - Trustpilot")

In [ ]:
healthygym_google_reviews['emotion_label'], healthygym_google_reviews['emotion_score'] = zip(*healthygym_google_reviews['Comment'].apply(classify_review))

# Display the dataset with the new columns
print(healthygym_google_reviews.head())

In [ ]:
#Group reviews by emotions - Google
healthygym_google_reviews_groupby_emotion_df = healthygym_google_reviews.groupby('emotion_label')['emotion_label'].count() #
healthygym_google_reviews_groupby_emotion_df = healthygym_google_reviews_groupby_emotion_df.sort_values(ascending=False)
healthygym_google_reviews_groupby_emotion_df = healthygym_google_reviews_groupby_emotion_df.reset_index(name="total_count")

# Get top 20 emotions
healthygym_google_reviews_groupby_emotion_df.head(20)

In [ ]:
# display the emotion histogram
ax = healthygym_google_reviews_groupby_emotion_df.plot.bar(x='emotion_label', y='total_count', rot=0)
ax.set_xlabel("emotion")
ax.set_ylabel("count")
ax.set_title("Predicted Emotions in Negative Reviews - Google")

In [ ]:
# Filter out the reviews that are from the top 30 bad review locations.
healthygym_trustpilot_bad_anger_list = healthygym_trustpilot_reviews[healthygym_trustpilot_reviews['emotion_label'] == "anger"]["Review Content"].to_list()
healthygym_google_bad_anger_list = healthygym_google_reviews[healthygym_google_reviews['emotion_label'] == "anger"]["Comment"].to_list()

#Merge the reviews to form a new list.
healthygym_combined_bad_anger_list = healthygym_trustpilot_bad_anger_list + healthygym_google_bad_anger_list

len(healthygym_combined_bad_anger_list)

In [ ]:
healthygym_combined_bad_anger_list[0]

In [ ]:
healthygym_combined_bad_anger_list_clean = clean_reviews(healthygym_combined_bad_anger_list)

In [ ]:
healthygym_combined_bad_anger_list_clean[0]

In [ ]:
# analyse the angry reviews
model_bad_anger = BERTopic(verbose=True)
model_bad_anger.fit(healthygym_combined_bad_anger_list_clean)
topic_bad_anger, probabilities_bad_anger = model_bad_anger.transform(healthygym_combined_bad_anger_list_clean)

In [ ]:
model_bad_anger.get_topic_freq().head(10)

In [ ]:
model_bad_anger.get_topic(0)

In [ ]:
model_bad_anger.get_topic(1)

In [ ]:
model_bad_anger.visualize_topics()

In [ ]:
model_bad_anger.visualize_barchart(topics=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
model_bad_anger.visualize_heatmap()

These are the top 10 word clusters from **negative** reviews with **angry** emotion. (changes slightly with consecutive runs).

*   weight bench machine equipment people
*   pas day pin bought email
*   been never walked past not
*   cancel membership refund email cancellation
*   parking car fine park free
*   shower cold temperature water hot
*   locker star room stolen staff
*   trainer personal rude member manager
*   toilet room soap changing sink
*   music loud class headphone noise

The general theme stays consistent with previous BERTopic analysis and the primary issue leading to negative and angry reviews are about weight and machine availability, access pass and pin, shower and toilet facilities, joining issues, parking fine, class and instructor availability and general noise and ambience in the gym.

# Using a large language model from Hugging Face

In [ ]:
!pip install accelerate
!pip install -U safetensors

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch
import accelerate

import pandas as pd
import numpy as np

In [ ]:
# Define the LLM model pipeline
model_llm = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_llm)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_llm,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:
sequences = pipeline(
    "In the following customer review, pick out the main 3 topics. Return them in a numbered list format, with each one on a new line.: I have paid my membership but I can't access the gym, I am still waiting on a code",
    max_length=1000,
    truncation=True,
    do_sample=True,
    top_k = 10,
    temperature=0.5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
  print(f"Result: {seq['generated_text']}")

In [ ]:
# This section is duplicate code from sections above to be able to run LLM without running the whole notepad
healthygym_trustpilot_df = pd.read_excel('/content/sample_data/Trustpilot_12_months.xlsx')
healthygym_google_df = pd.read_excel('/content/sample_data/Google_12_months.xlsx')
healthygym_google_df.dropna(inplace=True)

trustpilot_locations = healthygym_trustpilot_df['Location Name'].unique()
trustpilot_locations = trustpilot_locations.astype(str)
len(trustpilot_locations)

google_locations = healthygym_google_df["Club's Name"].unique()
google_locations = google_locations.astype(str)
len(google_locations)

intersection_of_locations = [ tp_location for tp_location in trustpilot_locations if tp_location in google_locations ]
len(intersection_of_locations)

In [ ]:
# This section is duplicate code from sections above to be able to run LLM without running the whole notepad
bad_healthygym_trustpilot_df = healthygym_trustpilot_df[healthygym_trustpilot_df["Review Stars"] < 3]
bad_healthygym_google_df = healthygym_google_df[healthygym_google_df["Overall Score"] < 3]

both_loc_healthygym_trustpilot_df = bad_healthygym_trustpilot_df[bad_healthygym_trustpilot_df['Location Name'].isin(intersection_of_locations)]
both_loc_healthygym_google_df = bad_healthygym_google_df[bad_healthygym_google_df["Club's Name"].isin(intersection_of_locations)]

bad_healthygym_trustpilot_list = both_loc_healthygym_trustpilot_df["Review Content"].to_list()
bad_healthygym_google_list = both_loc_healthygym_google_df["Comment"].to_list()

#Merge the reviews to form a new list.
all_bad_reviews_list = bad_healthygym_trustpilot_list + bad_healthygym_google_list

len(all_bad_reviews_list)

In [ ]:
topics_llm = []
print(len(all_bad_reviews_list))
iter = 0
for review in all_bad_reviews_list[0:1000]:
  if len(str(review)) < 1000:
    sequences = pipeline(
      f"In the following customer review, pick out the main 3 topics. Return them in a numbered list format, with each one on a new line. {review}",
      max_length=1000,
      truncation=True,
      do_sample=True,
      top_k = 10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
    )

    #print(sequences[0]['generated_text'])
    topic_splits = sequences[0]['generated_text'].splitlines()
    if len(topic_splits) > 1:
      topic_splits.pop(0)
    #print("topic_splits", topic_splits[0])

    for topic_split in topic_splits:
      topic_split = topic_split.replace("1.", "")
      topic_split = topic_split.replace("2.", "")
      topic_split = topic_split.replace("3.", "")
      topic_split = topic_split.replace("-", "")
      topic_split = topic_split.strip()
      topics_llm.append(topic_split)
  print(iter, "- ", len(all_bad_reviews_list))
  iter = iter + 1

print(topics_llm)
np_topics_llm = np.array(topics_llm)
np.save('np_topics_llm.npy', np_topics_llm)

In [ ]:
# Analyse the output of the LLM with BERTopic
model_negative_llm = BERTopic(verbose=True)
model_negative_llm.fit(topics_llm)
topic_model_negative_llm, probabilities_model_negative_llm = model_negative_llm.transform(topics_llm)

In [ ]:
model_negative_llm.get_topic_freq().head(10)

In [ ]:
model_negative_llm.get_topic(0)

In [ ]:
model_negative_llm.get_topic(1)

In [ ]:
model_negative_llm.visualize_topics()

In [ ]:
model_negative_llm.visualize_barchart(topics=[1,2,3,4,5,6,7,8,9,10])

In [ ]:
model_negative_llm.visualize_heatmap()

Topic clusters are as follows:

*  customer service active journey deliverycustomer
*  air conditioning atmosphere heating heat
*  busy crowded too overcrowding many
*  machines broken machine order always
*  experience app terrible unpleasant overall
*  access inability gym to enter
*  customer experience service unsatisfactory unsatisfied
*  parking car park details ticket
*  support contact lack phone lackluster
*  equipment apparatus moved cramped inappropriate

Although only 1000 negative reviews were processed, using LLM model falcon-7b-instruct, to return top 3 topics from each review, topic modelling using BERTopic resulted in almost the same word clusters as processing the reviews directly. Nevertheless, the clusters seem to better indicate the sentiment and the issues are clearer.

In [ ]:
loaded_arr = np.load('np_topics_llm.npy')
topics_llm = loaded_arr.tolist()

In [ ]:
topic_string = ', '.join(topics_llm[0:200])
print(topic_string)

In [ ]:
# Rerun LLM feeding the output from previous run, to get actionable insights
sequences_insights = pipeline(
  f"For the following text topics obtained from negative customer reviews, can you give some actionable insights that would help this gym company?. {topic_string}",
  max_length=3000,
  do_sample=True,
  top_k = 10,
  num_return_sequences=1,
  truncation=True,
  eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
sequences_insights

# Using Gensim

In [ ]:
# Install Gensim
!pip install gensim nltk datasets pyLDAvis ipykernel

In [ ]:
tokenized_docs = [doc.lower().split() for doc in all_bad_reviews_list_clean]

# Create a dictionary representation of the documents.
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_docs)

# Filter out words that occur fewer than 2 documents or more than 50% of the documents.
dictionary.filter_extremes(no_below=2, no_above=0.5)

# Create a BOW representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

# Set parameters.
num_topics = 10
passes = 20

In [ ]:
# Create the LDA model.
from gensim.models.ldamodel import LdaModel
lda_model = LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary, passes=passes)

In [ ]:
# Print the topics.
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic))

In [ ]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

# Prepare the visualisation.
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus, dictionary)
pyLDAvis.display(vis)

The top 10 topics from Gensim are as follows.

*   equipment, machine, time, people, busy, space, weight, get, enough, many
*   broken, machine, water, week, month, equipment, like, order, time, toilet
*   music, loud, hot, closed, even, open, class, water, work, day
*   shower, room, changing, toilet, dirty, cold, always, clean, locker, the
*   class, instructor, people, bike, cancelled, booked, minute, trainer, need, get
*   staff, member, manager, rude, customer, said, trainer, would, u, personal
*   weight, machine, equipment, staff, people, like, place, member, time, floor
*   air, year, staff, issue, member, con, cleaning, conditioning, equipment, hot
*   day, pas, pin, get, code, work, help, access, even, paid
*   time, membership, parking, month, get, day, email, i, pay, customer

The overall words in the topics are similar to the topics and words from other models like BERTopic and LLM.

# Conclusion

The reviews from Trustpilot and Google have been extensively analysed using various methods and models.

Initially the whole review from both Trustpilot and Google were analysed with histogram and word cloud. As the whole dataset was used, it did not provide any actionable insight. Subsequently the negative reviews were filtered out. The resultant histogram and word cloud started showing potential problem areas. However, it still did not provide a clear nature of the issue.

The negative reviews were then analysed with BERTopic. As the topic clusters were formed, the exact nature of the issues was coming to the fore. This was further finetuned by analysing based on location whereby negative reviews of top 30 locations were analysed. Emotion analysis was also performed on these and BERTopic was rerun on angry reviews.

Finally, the negative reviews were also analysed using a LLM model and Gensim. BERTopic analysis on output from the LLM model gave the clearest indication of the nature of the issues however due to resource constraints, the LLM could not provide any actionable insight. Gensim also provided similar topic clusters as those analysed by BERTopic.

Based on the overall analysis performed and explained above, the following are top issues highlighted by the reviews. HealthyGym is recommended to resolve these issues to gain competitive edge.

1.   Membership issue
* Sign up - access pass issue
* Cancellation issue
2. Too Busy
*   Equipment and weight availability
*   Trainer cancelling classed
*   Air conditioner/room temperature issue
3. Cleanliness
* Shower - mould and cold water
* Toilet
* Sink
4. Things getting stolen from locker room.
5. Rude staff.
6. Car park fine.
7. Loud music and general ambience.
8. Customer service

